In [1]:
from selenium import webdriver
import pandas as pd
import time
from tqdm import tqdm
import re

# 멜론 차트 크롤링

In [2]:
driver=webdriver.Chrome('/Users/gunwook/Desktop/chromedriver')

In [17]:
driver.get('https://www.melon.com/chart/search/index.htm')

In [234]:
chart_1 = driver.find_elements_by_css_selector('#lst50')

In [214]:
singer = []
sing = []
like = []
album = []

In [235]:
# 1위에서 50위까지
for i in range(0,50):
    a = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank02 > a')[0].text
    b = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div.ellipsis.rank01 > span > strong > a')[0].text
    c = int(chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(5) > div > button > span.cnt')[0].text.replace(',',''))
    d = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank03 > a')[0].text
    singer.append(a)
    sing.append(b)
    like.append(c)
    album.append(d)

In [236]:
# chart_2로 가기
driver.find_elements_by_css_selector('#frm > div.paginate.chart_page > span > a')[0].click()

In [237]:
chart_2 = driver.find_elements_by_css_selector('#lst100')

In [238]:
# 51위부터 100위까지
for i in range(0,50):
    a = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank02 > a')[0].text
    b = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div.ellipsis.rank01 > span > strong > a')[0].text
    c = int(chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(5) > div > button > span.cnt')[0].text.replace(',',''))
    d = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank03 > a')[0].text
    singer.append(a)
    sing.append(b)
    like.append(c)
    album.append(d)

In [240]:
driver.get('https://www.melon.com/search/total/index.htm?q=EXO&section=&searchGnbYn=Y&kkoSpl=Y&kkoDpType=&linkOrText=T&ipath=srch_form')

# 가온차트 크롤링

In [3]:
driver.get('http://www.gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL')

In [4]:
#chart_week_select > option:nth-child(120)
#chart_week_select > option:nth-child(3)
#chart_week_select > option:nth-child(107)

In [6]:
list1 = []
for i in tqdm(range(107, 2, -1)):
    # 주간 설정(range는 원하는만큼 지정)
    driver.find_elements_by_css_selector('#chart_week_select')[0].click()
    time = driver.find_elements_by_css_selector('#chart_week_select > option:nth-child({0})'.format(i))[0].text
    driver.find_elements_by_css_selector('#chart_week_select > option:nth-child({0})'.format(i))[0].click()
    
    
    rank = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr> td.ranking')
    chart_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.subject > p:nth-child(1)')
    gaon_100_score = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.count > p')
    production_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.production > p.pro')
    dist_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.production > p.dist')
    artist = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.subject > p.singer')
    for a,b,c,d,e,f in zip(chart_top_100,gaon_100_score,production_top_100,dist_top_100,artist,rank):
        
        a = re.split('[(-)]+' ,a.text)[0]
        e = e.text.split('|')[0]
        if e == '(여자)아이들':
            pass
        else:
            e = re.split('[(-)]+' ,e)[0]
        b = int(b.text.replace(',',''))
        list1.append([a, c.text, d.text, e, time, b, f.text])
        

100%|██████████| 105/105 [48:43<00:00, 27.85s/it]


In [7]:
df = pd.DataFrame(list1, columns = ['name','production','distributor','artist','time','score','rank'])

In [12]:
df.to_csv('가온차트.csv')

# 지니차트 크롤링

In [13]:
driver.get('https://www.genie.co.kr')

In [14]:
genre = []
runtime = []
sex = []
active_type = []

In [15]:
list2 = list(set(list(df['artist'] + ',' + df['name'])))

In [27]:
len(list2)

1454

In [28]:
df['name_artist'] = df['artist'] + ',' + df['name']

In [17]:
# 노래 정보 뽑아오기
# 장르, 런타임, 성별, 활동유형
for i in tqdm(set(list2)):
    try:
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text
        time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        if ':' in time:
            pass
        else:
            time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(5) > span.value').text
            gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        runtime.append(time)
        genre.append(gen)
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_elements_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.info > a.artist.ellipsis')[0].click()
        activity_type = driver.find_elements_by_css_selector('#body-content > div.artist-main-infos > div.info-zone > ul > li:nth-child(1) > span.value')[0].text.split('/')
        sex.append(activity_type[0])
        active_type.append(activity_type[1])
    except:
        i = i.split(',')
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text
        time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        if ':' in time:
            pass
        else:
            time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(5) > span.value').text
            gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        genre.append(gen)
        runtime.append(time)
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_elements_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.info > a.artist.ellipsis')[0].click()
        activity_type = driver.find_elements_by_css_selector('#body-content > div.artist-main-infos > div.info-zone > ul > li:nth-child(1) > span.value')[0].text.split('/')
        sex.append(activity_type[0])
        active_type.append(activity_type[1])


100%|██████████| 1454/1454 [1:04:54<00:00,  2.68s/it]


In [22]:
data1 = pd.DataFrame(genre,columns = ['장르'])

In [32]:
data = pd.DataFrame(list2,columns = ['조인'])

In [30]:
data2 = pd.DataFrame(runtime,columns = ['런타임'])
data3 = pd.DataFrame(sex,columns = ['성별'])
data4 = pd.DataFrame(active_type,columns = ['활동유형'])

In [34]:
final = pd.concat([data,data1,data2,data3,data4],axis= 1)

In [36]:
final = final.rename(columns = {'조인':'name_artist'})

In [38]:
df = pd.merge(df,final,how='left',on='name_artist')


In [188]:
df.to_csv('2019.06~2019.12.csv')

# 멜론 좋아요 뽑기

In [35]:
driver.get('https://www.melon.com/search/total/index.htm?q=%EC%82%AC%EB%9E%91%EC%97%90+%EC%97%B0%EC%8A%B5%EC%9D%B4+%EC%9E%88%EC%97%88%EB%8B%A4%EB%A9%B4&section=&linkOrText=T&ipath=srch_form')

In [36]:
likes = []

In [31]:
# 중간에 계속 팅기는 문제 발생. 
# 시간 텀 줘도 계속 팅기는 문제 생김
count = 0
for i in tqdm(list1):
    try:
        elem = driver.find_element_by_id('top_search')
        elem.clear()
        elem.send_keys(i)
        driver.find_element_by_css_selector('#header_wrap > div.wrap_search_field > fieldset > button.btn_icon.search_m > span').click()
        like = int(driver.find_element_by_css_selector('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > button > span.cnt').text.replace(',',''))
        likes.append(like)
        if count == 5:
            time.sleep(10)
            count = 0
        else:
            count += 1
    except:
        i = i.split(',')
        elem = driver.find_element_by_id('top_search')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_element_by_css_selector('#header_wrap > div.wrap_search_field > fieldset > button.btn_icon.search_m > span').click()
        like = int(driver.find_element_by_css_selector('#frm_searchSong > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > button > span.cnt').text.replace(',',''))
        likes.append(like)
        if count == 5:
            time.sleep(10)
            count = 0
        else:
            count += 1




  0%|          | 0/689 [00:00<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="top_search"]"}
  (Session info: chrome=81.0.4044.129)
